# Let's chat with a friend and I have a brain

Demo chat with Leolani that combine face recognition and storage in the BRAIN.

In [1]:
# general imports for EMISSOR and the BRAIN
import emissor as em
from cltl import brain
from cltl.triple_extraction.api import Chat, UtteranceHypothesis
from emissor.persistence import ScenarioStorage
from emissor.representation.annotation import AnnotationType, Token, NER
from emissor.representation.container import Index
from emissor.representation.scenario import Modality, ImageSignal, TextSignal, Mention, Annotation, Scenario
from cltl.brain.long_term_memory import LongTermMemory
from cltl.combot.backend.api.discrete import UtteranceType

# specific imports
import uuid
from datetime import datetime
import cv2
import requests
import pickle

#### The next utils are needed for the interaction and creating triples and capsules
import util.driver_util as d_util
import util.capsule_util as c_util
import util.face_util as f_util

[nltk_data] Downloading package punkt to /Users/piek/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Connect to the camera

In [2]:
### Link your camera
camera = cv2.VideoCapture(0)

### Initialise the BRAIN

In [3]:
# Initialise the brain in GraphDB
import pathlib
log_path=pathlib.Path('./logs')
print(type(log_path))
my_brain = brain.LongTermMemory(address="http://localhost:7200/repositories/sandbox",
                           log_dir=log_path,
                           clear_all=True)

<class 'pathlib.PosixPath'>
2021-10-29 15:22:30,706 -    DEBUG -    cltl.brain.basic_brain.LongTermMemory - Booted


2021-10-29 15:22:30.706 DEBUG basic_brain - __init__: Booted


2021-10-29 15:22:30,708 -    DEBUG -    cltl.brain.basic_brain.LongTermMemory - Clearing brain


2021-10-29 15:22:30.708 DEBUG basic_brain - clear_brain: Clearing brain


2021-10-29 15:22:33,072 -    DEBUG -    cltl.brain.basic_brain.LongTermMemory - Checking if ontology is in brain


...

2021-10-29 15:22:36.928 DEBUG basic_brain - __init__: Booted


2021-10-29 15:22:37,223 -    DEBUG -   cltl.brain.basic_brain.TrustCalculator - Posting query


2021-10-29 15:22:37.223 DEBUG basic_brain - _submit_query: Posting query


2021-10-29 15:22:37,269 -     INFO -   cltl.brain.basic_brain.TrustCalculator - Computed trust for all known agents


2021-10-29 15:22:37.269 INFO trust_calculator - compute_trust_network: Computed trust for all known agents


## Standard initialisation of a scenario

In [4]:
from random import getrandbits

##### Setting the location
place_id = getrandbits(8)
location = requests.get("https://ipinfo.io").json()

##### Setting the agents
AGENT = "Leolani2"
HUMAN_NAME = "Stranger"
HUMAN_ID = "stranger"

### The name of your scenario
scenario_id = datetime.today().strftime("%Y-%m-%d-%H:%M:%S")

### Specify the path to an existing data folder where your scenario is created and saved as a subfolder
scenario_path = "./data"

### Define the folder where the images are saved
imagefolder = scenario_path + "/" + scenario_id + "/" + "image"


### Create the scenario folder, the json files and a scenarioStorage and scenario in memory
scenarioStorage = d_util.create_scenario(scenario_path, scenario_id)
scenario = scenarioStorage.create_scenario(scenario_id, datetime.now().microsecond, datetime.now().microsecond, AGENT)

Directory  ./data/2021-10-29-15:22:42  Created 
Directory  ./data/2021-10-29-15:22:42/image  Created 


## Loading the docker containers for face detection and face property detection

#### If the docker images are running you can skip the next part

You only need to load the dockers once. The first time you load the docker, the images will be donwloaded from the DockerHub. This may take a few minutes depending on the speed of the internet connection. The images are cached in your local Docker installation.

One the images are in your local Docker, they are loaded instantaniously. Once the docker is started you do not need to start it again and you can skip the next commands.

In [5]:
#container_fdr = f_util.start_docker_container("tae898/face-detection-recognition:v0.1", 10002)
#container_ag = f_util.start_docker_container("tae898/age-gender:v0.2", 10003)

If there is a problem starting the dockers, you may need to kill them and start them again. Use the following command to kill and rerun the previous command. Note that if there are running already you should not restart. Starting it again gives an error that the port is occupied.

In [6]:
#!docker kill $(docker ps -q)

## Extracting and storing triples from an utterance

The next function processes any textSignal using the baseline language model from the Leolani platform.
This model uses a context-free grammar and closed-class lexicons specifically designed for social robot interaction.

The function creates a capsule for posting any triples and perspective to the BRAIN.
If no triples are extracted, a dummy prompt is returned: "Any gossip?".

Any thoughts coming from the BRAIN are returned.

In [7]:
def process_text_and_think (scenario: Scenario, 
                  place_id:str, 
                  location: str, 
                  textSignal: TextSignal,
                  human_id: str,
                  my_brain:LongTermMemory):
    thoughts = ""
    chat = Chat(human_id)
    chat.add_utterance([UtteranceHypothesis(c_util.seq_to_text(textSignal.seq), 1.0)])
    chat.last_utterance.analyze()
    # No triple was extracted, so we missed three items (s, p, o)
    if chat.last_utterance.triple is None:
        utterance = "Any gossip?" + '\n'
    else:
        triple = c_util.rephrase_triple_json_for_capsule(chat.last_utterance.triple)
        # A triple was extracted so we compare it elementwise
        capsule = c_util.scenario_utterance_and_triple_to_capsule(scenario, 
                                                                  place_id,
                                                                  location,
                                                                  textSignal, 
                                                                  human_id,
                                                                  chat.last_utterance.perspective, 
                                                                  triple)
        print('Capsule:', capsule)
        thoughts = my_brain.update(capsule, reason_types=True)
        #print(thoughts)
    return thoughts    

## Create a new friend.

The next functions are needed to get the properties and visual information for identifying a new friend.
The visual information is based on the camera images of the uses from which we extract an averaged embedding.
These embeddings are store in the *friend_embeddings* folder. 

By comparing an image with the stored embeddings, the system decides whether a person is a *stranger*.
In case the user is a *stranger*, the system will try to get to know him/her.

If you delete someone's embeddings from the *friend_embeddings* folder. This person will become a *stranger* again.

In [8]:
### Function that creates capsules for the basic properties of a friend: name, age and gender.
### The capsules are sent to the BRAIN. Thoughts are caught and returned for each property.

def process_new_friend_and_think (scenario: Scenario, 
                  place_id:str, 
                  location: str, 
                  human_id: str,
                  textSignal: TextSignal,
                  imageSignal: ImageSignal,
                  age: str,
                  gender: str,
                  human_name: str,
                  my_brain:LongTermMemory):
    age_thoughts = ""
    gender_thoughts = ""
    name_thoughts = ""


    if human_name:
        # A triple was extracted so we compare it elementwise
        perspective = {"certainty": 1, "polarity": 1, "sentiment": 1}
        capsule = c_util.scenario_utterance_to_capsule(scenario, 
                                                                  place_id,
                                                                  location,
                                                                  textSignal,
                                                                  human_id,
                                                                  perspective,
                                                                  human_id,
                                                                  "label",
                                                                  human_name)

        name_thoughts = my_brain.update(capsule, reason_types=True, create_label=False)
        print('Name capsule:', capsule)


    if age:
        # A triple was extracted so we compare it elementwise
        capsule = c_util.scenario_image_triple_to_capsule(scenario, 
                                                                  place_id,
                                                                  location,
                                                                  imageSignal,
                                                                  "front_camera", 
                                                                  human_id,
                                                                  "age",
                                                                  str(age))

        age_thoughts = my_brain.update(capsule, reason_types=True, create_label=False)
        print('Age capsule:', capsule)

    if gender:
        capsule = c_util.scenario_image_triple_to_capsule(scenario, 
                                                                  place_id,
                                                                  location,
                                                                  imageSignal,
                                                                  "front_camera", 
                                                                  human_id,
                                                                  "gender",
                                                                  gender)

        gender_thoughts = my_brain.update(capsule, reason_types=True, create_label=False)
        print('Gender capsule:', capsule)

    return name_thoughts, age_thoughts, gender_thoughts    

In [9]:
### Function that tries to get the name for a new person. A while is used till the user is happy.
### From the name a unique ID *human_id* is created by adding the time_stamp
### We store the face embeddings in the friend_embeddings folder using the unique ID *human_id*
### The function returns the human_name and the human_id.
### Human_name is used to address the user, and human_id is used to store properties of the user

def get_to_know_person(scenario: Scenario, agent:str, gender:str, age: str, uuid_name: str, embedding):
        ### This is a stranger
        ### We create the agent response and store it as a text signal
        human_name = "Stranger"
        response = (
            f"Hi there. We haven't met. I only know that \n"
            f"your estimated age is {age} \n and that your estimated gender is "
            f"{gender}. What's your name?"
        )
        print(f"{agent}: {response}")
        textSignal = d_util.create_text_signal(scenario, response)
        scenario.append_signal(textSignal)
        
        confirm = ""
        while confirm.lower().find("yes")==-1:
            ### We take the response from the user and store it as a text signal
            utterance = input("\n")
            textSignal = d_util.create_text_signal(scenario, utterance)
            scenario.append_signal(textSignal)
            print(utterance)
            #### We hack the response to find the name of our new fiend
            #### This name needs to be set in the scenario and assigned to the global variable human
            human_name = " ".join([foo.title() for foo in utterance.strip().split()])
            human_name = "_".join(human_name.split())
        
            response = (f"So your name is {human_name}?")
            print(f"{agent}: {response}")
            textSignal = d_util.create_text_signal(scenario, response)
            scenario.append_signal(textSignal)
            
            ### We take the response from the user and store it as a text signal
            confirm = input("\n")
            textSignal = d_util.create_text_signal(scenario, confirm)
            scenario.append_signal(textSignal)


        current_time = str(datetime.now().microsecond)
        human_id = human_name+"_t_"+current_time
        #### We create the embedding
        to_save = {"uuid": uuid_name["uuid"], "embedding": embedding}

        with open(f"./friend_embeddings/{human_id}.pkl", "wb") as stream:
            pickle.dump(to_save, stream)
            
        return human_id, human_name, textSignal

Friends of Leolani are saved in the embeddings folder

In [10]:
#First step is to identify the speaker

imagepath = ""
uuid_name = ""
# First signal
success, frame = camera.read()

if success:
    ### check if we know the human
    current_time = str(datetime.now().microsecond)
    imagepath = imagefolder + "/" + current_time + ".png"
    cv2.imwrite(imagepath, frame)
    imageSignal = d_util.create_image_signal(scenario, imagepath)
    (
        genders,
        ages,
        bboxes,
        faces_detected,
        det_scores,embeddings,
    ) = f_util.do_stuff_with_image(imagepath)

    # Here we assume that only one face is in the image
    # TODO: deal with multiple people.
    for k, (gender, age, bbox, uuid_name, faceprob, embedding) in enumerate(
        zip(genders, ages, bboxes, faces_detected, det_scores, embeddings)
    ):
        age = round(age["mean"])
        gender = "male" if gender["m"] > 0.5 else "female"
        bbox = [int(num) for num in bbox.tolist()]
    assert k == 0
    if uuid_name["name"] is None:
        ### This is a stranger
        ### We call the get_to_know function to create the data for a new friend
        HUMAN_ID, HUMAN_NAME, textSignal = get_to_know_person(scenario, AGENT, gender, age, uuid_name, embedding)
        ### We store the data in the BRAIN for the human_id that is returned            
        name_thoughts, age_thoughts, gender_thoughts = process_new_friend_and_think (scenario, 
                  place_id, 
                  location, 
                  HUMAN_ID,
                  textSignal,
                  imageSignal,
                  age,
                  gender,
                  HUMAN_NAME,
                  my_brain)
        
        ### The system responds to the processing of the new name input and stores it as a textsignal
        print(AGENT + f": Nice to meet you, {HUMAN_NAME}")
        response = f": Nice to meet you, {HUMAN_NAME}"
        textSignal = d_util.create_text_signal(scenario, response)
        scenario.append_signal(textSignal)

    else:
        ### We know this person
        HUMAN_ID= uuid_name['name']
        HUMAN_NAME = HUMAN_ID.split("_t_")[0]
        response = f"Hi {HUMAN_NAME}. Nice to see you again :)"
        print(f"{AGENT}: {response}")
        textSignal = d_util.create_text_signal(scenario, response)
        scenario.append_signal(textSignal)


2021-10-29 15:22:48.633 INFO face_util - load_binary_image: ./data/2021-10-29-15:22:42/image/592760.png image loaded!
2021-10-29 15:22:49.365 INFO face_util - run_face_api: got <Response [200]> from server!...


TypeError: scalar() argument 1 must be numpy.dtype, not _IDProxy

## Proceed to communicate with an identified friend

The next while loop is for continuous communication with the identified user until *stop* or *bye*.
We process the user input and the captured image sequentially step by setp in the code block of the while loop.
At the end of the code block, a response is generated and new input from the user is requested.

In [13]:
### First prompt
response = "How are you doing "+HUMAN_NAME
textSignal = d_util.create_text_signal(scenario, response)
scenario.append_signal(textSignal)

print(AGENT + ": " + response)

#### Get the input from the user
utterance = input("\n")
print(HUMAN_NAME + ": " + utterance)

while not (utterance.lower() == "stop" or utterance.lower() == "bye"):
    textSignal = d_util.create_text_signal(scenario, utterance)
    scenario.append_signal(textSignal)

    # @TODO: also annotate the textSignal
    # Apply some processing to the textSignal and add annotations
        
    # We process the utterance and store triples in the brain
    # We catch the thoughts as the response
    thoughts = process_text_and_think (scenario, 
                  place_id, 
                  location, 
                  textSignal, 
                  HUMAN_ID, my_brain)
        
    ## We capture the image again
    ## For now, we only take pictures of faces of the user
    ## @TODO add object recognition and check if there are other people detected than the user
   
    success, frame = camera.read()
    if success:
        current_time = str(datetime.now().microsecond)
        imagepath = imagefolder + "/" + current_time + ".png"
        cv2.imwrite(imagepath, frame)
        (
            genders,
            ages,
            bboxes,
            faces_detected,
            det_scores,
            embeddings,
        ) = f_util.do_stuff_with_image(imagepath)



    if success:
        imageSignal = d_util.create_image_signal(scenario, imagepath)
        container_id = str(uuid.uuid4())

        #### Properties are now stored as annotations
        #### We do not store these proeprties again to the BRAIN
        for gender, age, bbox, name, faceprob in zip(
            genders, ages, bboxes, faces_detected, det_scores
        ):

            age = round(age["mean"])
            gender = "male" if gender["m"] > 0.5 else "female"
            bbox = [int(num) for num in bbox.tolist()]

        f_util.add_face_annotation(imageSignal,container_id, "front_camera", container_id, current_time,
                                        bbox,HUMAN_ID,HUMAN_NAME, age, gender,faceprob)
                                   
        scenario.append_signal(imageSignal)
        
        ### We created a perceivedBy triple for this experience, 
        ### @TODO we need to include the bouding box somehow in the object
        capsule = c_util.scenario_image_triple_to_capsule(scenario, 
                                                                  place_id,
                                                                  location,
                                                                  imageSignal,
                                                                  "front_camera", 
                                                                  human_id,
                                                                  "perceivedBy",
                                                                  imageSignal.id)

         response = my_brain.update(capsule, reason_types=True, create_label=False) ## use this version if you want to use the URI as subject

    # Create the response from the system and store this as a new signal
    # We could use the throughts to respond
    # @TODO generate a response from the thoughts or based on the user query

    utterance = "So you what do you want to talk about " + HUMAN_NAME + "\n"
    response = utterance[::-1]
    print(AGENT + ": " + utterance)
    textSignal = d_util.create_text_signal(scenario, utterance)
    scenario.append_signal(textSignal)

    # Getting the next input signals
    utterance = input("\n")

    ### We now have a new input check if the user wants to continue

Leolani2: How are you doing Piek



 I am great.


Piek: I am great.
2021-10-29 10:23:32,112 -     INFO - cltl.triple_extraction.api.Chat (Piek_t_191694)     000 - << Start of Chat with Piek_t_191694 >>


2021-10-29 10:23:32.112 INFO api - __init__: << Start of Chat with Piek_t_191694 >>


2021-10-29 10:23:32,114 -     INFO -               cltl.triple_extraction.api - Started POS tagger


...

2021-10-29 10:23:46.634 INFO api - __init__: << Start of Chat with Piek_t_191694 >>


2021-10-29 10:23:47,475 -     INFO - cltl.triple_extraction.api.Chat (Piek_t_191694)     001 - Piek_t_191694: "Universe and about you"


2021-10-29 10:23:47.475 INFO api - add_utterance: Piek_t_191694: "Universe and about you"
2021-10-29 10:23:47.477 WARNING analyzer - analyze: Couldn't parse input
2021-10-29 10:23:47.543 INFO face_util - load_binary_image: ./data/2021-10-29-10:19:30/image/510434.png image loaded!
2021-10-29 10:23:48.296 INFO face_util - run_face_api: got <Response [200]> from server!...
2021-10-29 10:23:48.297 INFO face_util - run_face_api: 1 faces deteced!
2021-10-29 10:23:48.342 INFO face_util - run_age_gender_api: got <Response [200]> from server!...


Leolani2: So you what do you want to talk about Piek




 stop


### Set the end time of the scenario, save it and stop the containers

After we stopped the interaction, we set the end time and save the scenario as EMISSOR data.

In [14]:
#scenario.scenario.end = datetime.now().microsecond
scenarioStorage.save_scenario(scenario)

In [15]:
### Stopping the docker containers
### This is only needed of you started them in this notebook

#f_util.kill_container(container_fdr)
#f_util.kill_container(container_ag)

In [16]:
#### Stop the camera when we are done
camera.release()

## End of notebook